# Project Report
The project is open-ended with the exact goal to be decided amongst your group but the theme is around simulations and data relating to the coronavirus epidemic. You should make a program or several programs that can either run simulations or produce plots based on real data.

Along with your code there should be a group Project Report explaining your code and what it does and including some figures that are output from your code or perhaps links to videos or any other output.

The report should not be long. The purpose of the report is to succinctly explain any models you are using or data that you have obtained and to show the resulting plots that you have made. This project is about code (rather than modelling, data analysis, or epidemiology) so the report should be about showcasing some outputs of your code and explaining how your code can be used to produce those as well as other outputs. A minimum of explanation is needed for those outputs to be understandable but you do not need to have a "background" section or a "literature review" or anything like that - It would be reasonable to include links to any references though e.g. if your models are inspired by something else.

# Data
Carl writes where he got the data from